In [32]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, TimeDistributed, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop, Adam

train_data = pd.read_csv('../data/balanced_train.csv')
test_data = pd.read_csv('../data/balanced_test.csv')

genre_columns = train_data.columns.drop(['Name', 'Description Tokenized'])

# Tokenizar os textos
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(train_data['Description Tokenized']) + list(test_data['Description Tokenized']))
word_index = tokenizer.word_index
num_words = len(word_index) + 1

X_train = tokenizer.texts_to_sequences(train_data['Description Tokenized'])
X_test = tokenizer.texts_to_sequences(test_data['Description Tokenized'])

maxlen = max(max([len(sequence) for sequence in X_train]), max([len(sequence) for sequence in X_test]))

# Padronizar os textos
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

#labels one-hot encoded
y_train = train_data[genre_columns].values
y_test = test_data[genre_columns].values


# modelo LSTM unidirecional
embedding_dim = 100

model = Sequential()
model.add(Embedding(num_words, embedding_dim, input_length=maxlen))
model.add(LSTM(128, return_sequences=True, dropout=0.25, recurrent_dropout=0.25))
model.add(TimeDistributed(Dense(128, activation='relu')))
model.add(Dropout(0.5))
model.add(LSTM(128, dropout=0.25, recurrent_dropout=0.25))
model.add(Dense(len(genre_columns), activation='sigmoid'))

# Compilar e treinar o modelo
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

num_epochs = 15
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test), verbose=2)

# 4. Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

Epoch 1/15
160/160 - 95s - loss: 0.5641 - accuracy: 0.2596 - val_loss: 0.5592 - val_accuracy: 0.2648 - 95s/epoch - 597ms/step
Epoch 2/15
160/160 - 68s - loss: 0.5580 - accuracy: 0.2645 - val_loss: 0.5586 - val_accuracy: 0.2648 - 68s/epoch - 428ms/step
Epoch 3/15
160/160 - 71s - loss: 0.5575 - accuracy: 0.2577 - val_loss: 0.5590 - val_accuracy: 0.2648 - 71s/epoch - 444ms/step
Epoch 4/15
160/160 - 67s - loss: 0.5547 - accuracy: 0.2586 - val_loss: 0.5443 - val_accuracy: 0.2219 - 67s/epoch - 421ms/step
Epoch 5/15
160/160 - 65s - loss: 0.5275 - accuracy: 0.2827 - val_loss: 0.5049 - val_accuracy: 0.3023 - 65s/epoch - 408ms/step
Epoch 6/15
160/160 - 61s - loss: 0.4634 - accuracy: 0.3491 - val_loss: 0.4736 - val_accuracy: 0.2570 - 61s/epoch - 380ms/step
Epoch 7/15
160/160 - 65s - loss: 0.4152 - accuracy: 0.3735 - val_loss: 0.4414 - val_accuracy: 0.3805 - 65s/epoch - 406ms/step
Epoch 8/15
160/160 - 72s - loss: 0.3754 - accuracy: 0.4218 - val_loss: 0.4377 - val_accuracy: 0.3711 - 72s/epoch - 447

In [33]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
y_pred_binarized = np.round(y_pred)

report = classification_report(y_test, y_pred_binarized, target_names=genre_columns, zero_division=0)
print(report)

40/40 [==============================] - 6s 133ms/step
                         precision    recall  f1-score   support

                 Comedy       0.69      0.72      0.71       487
                  Crime       0.73      0.79      0.76       271
                  Drama       0.71      0.82      0.76       613
                Romance       0.71      0.64      0.67       270
   Action and Adventure       0.76      0.65      0.70       463
Documentary and History       0.77      0.50      0.61       172
   Family and Animation       0.74      0.60      0.66       275
     Fantasy and Sci-Fi       0.71      0.64      0.67       259
    Horror and Thriller       0.68      0.61      0.64       309

              micro avg       0.72      0.69      0.70      3119
              macro avg       0.72      0.66      0.69      3119
           weighted avg       0.72      0.69      0.70      3119
            samples avg       0.71      0.67      0.67      3119

